In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import MultipleLocator
# plt.style.use('seaborn-paper')
import seaborn as sns
import joblib

# plt.rcParams['font.family']='Times New Roman,Microsoft YaHei'# 设置字体族，中文为微软雅黑，英文为Times New Roman
plt.rcParams['font.sans-serif'] = 'Times New Roman'
plt.rcParams['mathtext.fontset'] = 'stix'  # 设置数学公式字体为stix
plt.rcParams["text.usetex"] = False
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
import statsmodels.api as sm
import warnings

warnings.filterwarnings('ignore', category=FutureWarning)
from sklearn.metrics import mean_squared_error


In [2]:

datajh = pd.read_csv("groupedjhS.csv")
datacm = pd.read_csv("groupedcmS.csv")
datajhsoa = pd.read_csv("groupedjhSOA.csv")
datacmsoa = pd.read_csv("groupedcmSOA.csv")

In [3]:
datajhsoa["SOA"] = 0
datajh["SOA"] = 0
for i in datajhsoa.columns[(datajhsoa.columns.get_loc("0.25um")):(datajhsoa.columns.get_loc("0.28um"))]:
    datajhsoa["SOA"] = datajhsoa["SOA"] + datajhsoa[i]
    datajh["SOA"] = datajh["SOA"] + datajh[i]
datajhsoa["CSOA"] = 0
for i in datajhsoa.columns[(datajhsoa.columns.get_loc("C0.25um")):(datajhsoa.columns.get_loc("C0.28um"))]:
    datajhsoa["CSOA"] = datajhsoa["CSOA"] + datajhsoa[i]
datacmsoa["SOA"] = 0
datacm["SOA"] = 0
for i in datacmsoa.columns[(datacmsoa.columns.get_loc("0.25um")):(datacmsoa.columns.get_loc("0.28um"))]:
    datacmsoa["SOA"] = datacmsoa["SOA"] + datacmsoa[i]
    datacm["SOA"] = datacm["SOA"] + datacm[i]
datacmsoa["CSOA"] = 0
for i in datacmsoa.columns[(datacmsoa.columns.get_loc("C0.25um")):(datacmsoa.columns.get_loc("C0.28um"))]:
    datacmsoa["CSOA"] = datacmsoa["CSOA"] + datacmsoa[i]

In [4]:
datare = datajh.groupby(["Hour_Min"]).mean(numeric_only=True).reset_index()

In [6]:
X=datare.异戊二烯浓度
Z=datare.地面层温度
y=datare.SOA

In [7]:
import statsmodels.api as sm
from statsmodels.sandbox.regression.gmm import IV2SLS
# 进行2SLS回归
model = IV2SLS(y, X, Z)
results = model.fit()

In [8]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                          IV2SLS Regression Results                           
==============================================================================
Dep. Variable:                    SOA   R-squared:                       0.878
Model:                         IV2SLS   Adj. R-squared:                  0.878
Method:                     Two Stage   F-statistic:                       nan
                        Least Squares   Prob (F-statistic):                nan
Date:                Wed, 23 Oct 2024                                         
Time:                        15:41:16                                         
No. Observations:                1440                                         
Df Residuals:                    1439                                         
Df Model:                           1                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
异戊二烯浓度         0.8295      0.008    108.050      0.000       0.814       0.845
==============================================================================
Omnibus:                      243.973   Durbin-Watson:                   0.979
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              543.812
Skew:                          -0.957   Prob(JB):                    8.18e-119
Kurtosis:                       5.324   Cond. No.                         1.00
==============================================================================
"""

In [9]:
# OLS 回归
model_ols = sm.OLS(y, X).fit()

# 2SLS 回归
model_iv = IV2SLS(y, X, Z).fit()

# 比较OLS和IV回归的系数差异（Hausman检验）
b_ols = model_ols.params
b_iv = model_iv.params
V_ols = model_ols.cov_params()
V_iv = model_iv.cov_params()

# Hausman 检验统计量
diff = b_iv - b_ols
stat = diff.T @ np.linalg.inv(V_iv - V_ols) @ diff

print("Hausman 检验统计量: ", stat)

Hausman 检验统计量:  640.3730710223003


In [10]:
# 进行2SLS回归
model_iv = IV2SLS(y, X, Z).fit()

# 提取回归残差
residuals = model_iv.resid

# Sargan 检验统计量：残差与工具变量的回归
sargan_stat = len(y) * sm.OLS(residuals, Z).fit().rsquared

print("Sargan 检验统计量: ", sargan_stat)

Sargan 检验统计量:  -3.197442310920451e-13
